In [1]:
import os

In [2]:
%pwd

'd:\\Python_DataScience\\Wine_Quality_Prediction\\Wine_Quality_Prediction_End_To_End_ML\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'd:\\Python_DataScience\\Wine_Quality_Prediction\\Wine_Quality_Prediction_End_To_End_ML'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from wine_pred.constants import *

from wine_pred.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__ (
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config




In [17]:
from wine_pred.logging.logger import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [22]:
class DataTransformation:
    def __init__ (self, config: DataTransformationConfig):
        self.config = config
        self.log_columns = ['fixed acidity', 'volatile acidity','residual sugar', 'citric acid', 'chlorides', 'sulphates', 'free sulfur dioxide', 'total sulfur dioxide', 'alcohol']

    def data_transformation(self):
        preprocessor = ColumnTransformer(
            transformers=[
            ('quantile', QuantileTransformer(output_distribution='normal'), self.log_columns),
            ],
        remainder='passthrough')

        df = pd.read_csv(self.config.data_path)
        df_transformed = preprocessor.fit_transform(df)
    
        # Convert the transformed array back to a DataFrame
        df_transformed = pd.DataFrame(df_transformed, columns=df.columns)
    
        return df_transformed
        
    def data_train_test_split(self):

        train, test = train_test_split(self.data_transformation(),test_size=0.25)
        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"), index=False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

In [23]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.data_transformation()
    data_transformation.data_train_test_split()
    
except Exception as e:
    raise e

[ 2024-07-06 10:41:43,698 ] 29 root - INFO - yaml file: config\config.yaml loaded succesfully
[ 2024-07-06 10:41:43,699 ] 29 root - INFO - yaml file: params.yaml loaded succesfully
[ 2024-07-06 10:41:43,701 ] 29 root - INFO - yaml file: schema.yaml loaded succesfully
[ 2024-07-06 10:41:43,702 ] 43 root - INFO - Created directory at: artifacts
[ 2024-07-06 10:41:43,703 ] 43 root - INFO - Created directory at: artifacts/data_transformation
[ 2024-07-06 10:41:43,742 ] 27 root - INFO - Splited data into training and test sets
[ 2024-07-06 10:41:43,742 ] 28 root - INFO - (857, 13)
[ 2024-07-06 10:41:43,742 ] 29 root - INFO - (286, 13)
